In [2]:
import os
import sys
import csv
import gc
import glob
import shutil
import time
import re
import random
import numpy as np
import pandas as pd
from string import punctuation
from collections import Counter, OrderedDict
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
import keras
from keras import Model
from keras.layers import *
from keras.models import *
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.regularizers import *
from keras import backend as K
from keras.legacy import interfaces
from keras.engine.topology import Layer
from keras.layers.merge import _Merge
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.generic_utils import serialize_keras_object
from keras.utils.generic_utils import deserialize_keras_object
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, StratifiedShuffleSplit

Using TensorFlow backend.


In [3]:
np.random.seed(2018)
tf.set_random_seed(2018)

In [7]:
# Whether to remove punctuation
drop = False
max_features = 90000
maxlen = 72
cv = True

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("Train shape: ", train.shape)
print("Test shape: ", test.shape)

OSError: Initializing from file failed

In [4]:
# Update by find_contractions analysis
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because",
                "could've": "could have", "couldn't": "could not", "didn't": "did not",
                "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not",
                "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is",
                "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have",
                "i'm": "i am", "i've": "I have", "i'd": "i would", "i'd've": "i would have",
                "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have",
                "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will",
                "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not",
                "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
                "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
                "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
                "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
                "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
                "y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                "you'll've": "you will have", "you're": "you are", "you've": "you have" }

# Update by miswords analysis
mispell = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling',
           'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor',
           'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
           'qoura': 'quora', 'sallary': 'salary', 'whta': 'what', 'narcisist': 'narcissist',
           'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much',
           'howmany': 'how many', 'whydo': 'why do', 'doI': 'do i', 'theBest': 'the best', 'howdoes': 'how does',
           'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating',
           'pennis': 'penis', 'etherium': 'ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data',
           '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend',
           'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization',
           'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def sep_punc(x):
    # ’标点符号‘变为‘ 标点符号 ’
    x = str(x)
    for p in puncs:
        x = x.replace(p, f' {p} ')
    return x

def clean_numbers(x):
    """
    hmm why is "##" in there? Simply because as a reprocessing all numbers bigger 
    tha 9 have been replaced by hashs. I.e. 15 becomes ## while 123 becomes ### or 
    15.80€ becomes ##.##€. So lets mimic this preprocessing step to further improve our embeddings coverage
    """
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [ ]:
tmp = []
for x in tqdm(train.question_text):
    for c in x:
        if not c.isalnum():
            tmp.append(c)
for x in tqdm(test.question_text):
    for c in x:
        if not c.isalnum():
            tmp.append(c)
puncs = set(tmp)

In [ ]:
train["question_text"] = train["question_text"].str.lower()
test["question_text"] = test["question_text"].str.lower()
print("Lower done")

# 两个字典并集
mispell_dict = dict(set(contractions.items()) | set(mispell.items()))
mispellings, mispellings_re = _get_mispell(mispell_dict)
train["question_text"] = train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
print("Clean speelings done")

train["question_text"] = train["question_text"].progress_apply(lambda x: sep_punc(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: sep_punc(x))
print("Sep punc done")

train["question_text"] = train["question_text"].progress_apply(lambda x: clean_numbers(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_numbers(x))
print("Clean numbers done")

In [ ]:
train_X = train['question_text']
# filters注意设置，因为标点符号被保留
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(train_X)
train_X = tokenizer.texts_to_sequences(train_X)
train_X = pad_sequences(train_X, maxlen=maxlen)

test_X = test['question_text']
test_X = tokenizer.texts_to_sequences(test_X)
test_X = pad_sequences(test_X, maxlen=maxlen)
train_y = train['target'].values
# 通过word_index与Embedding实现一一对应
word_index = tokenizer.word_index

In [ ]:
glove_path = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
para_path = "../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt"
fasttext_path = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
def load_embed(max_features, word_index,embed_path,embed_type):
    if embed_type == 'para':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(embed_path, encoding="utf8", errors='ignore') if len(o)>100)
    elif embed_type == 'glove':
        embeddings_index = dict(get_coefs(*v.split(" ")) for v in open(embed_path))
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(embed_path) if len(o)>100)
        
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
embed_glove = load_embed(max_features, word_index, glove_path,'glove')

In [ ]:
embed_para = load_embed(max_features, word_index, para_path,'para')
glove_para = np.mean([embed_glove,embed_para], axis=0)

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
class LstmAtn():
    def model(self, embedding_matrix, maxlen, max_features):
        inp = Input(shape=(maxlen,))
        emb_size = embedding_matrix.shape[1]
        x_emb = Embedding(max_features, emb_size, weights=[embedding_matrix], trainable=True)(inp)
        x = SpatialDropout1D(0.5)(x_emb)
        x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
        y = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)

        atn_1 = Attention(maxlen)(x)
        atn_2 = Attention(maxlen)(y)
        avg_pool = GlobalAveragePooling1D()(y)
        max_pool = GlobalMaxPooling1D()(y)
        x = concatenate([atn_1, atn_2, avg_pool, max_pool])
#         x = BatchNormalization()(x)
        x = Dense(16, activation="relu")(x)
        x = Dropout(0.1)(x)
#         x = BatchNormalization()(x)
        output = Dense(1, activation="sigmoid")(x)

        model = Model(inputs=inp, outputs=output)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
    
def ConvBlock(inp,filters):
    x = Conv1D(filters, 3)(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(filters, 3)(x)
    x = BatchNormalization()(x)
    outp = Activation('relu')(x)
    return outp

class VDNN():
    def model(self, embedding_matrix, maxlen, max_features):
        inp = Input(shape=(maxlen,))
        emb_size = embedding_matrix.shape[1]
        x_emb = Embedding(max_features, emb_size, weights=[embedding_matrix], trainable=True)(inp)
        
        x = Conv1D(64, 3, activation='relu')(x_emb)
        x = ConvBlock(x, 256)
        
        x = MaxPooling1D(2)(x)
        x = ConvBlock(x, 256)
        
        x = MaxPooling1D(2)(x)
        x = ConvBlock(x, 256)
        
        x = Dense(512)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        output = Dense(1, activation="sigmoid")(x)

        model = Model(inputs=inp, outputs=output)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model
        


In [ ]:
# def train_offline():
#     model = Model().BiGRUAtt(embed_glove, maxlen, max_features)
# filepath = "BiGRUAtt.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min')
keras_f1 = Metrics()
# clr = CyclicLR(base_lr=0.001, max_lr=0.006,
#                     step_size=2000., mode='exp_range',
#                     gamma=0.99994)
callbacks = [keras_f1]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2,random_state=2018, stratify=train_y)


In [ ]:
model.summary()

In [ ]:
model = VDNN().model(embed_glove, maxlen, max_features)


In [ ]:
model.fit(X_train, y_train, batch_size=512, epochs=5, validation_data=(X_val, y_val),)

In [ ]:
history = model.fit(X_train, y_train, batch_size=512, epochs=5, validation_data=(X_val, y_val),)
pred_glove_val_y = model.predict([X_val], batch_size=1024, verbose=1)

In [ ]:
from sklearn import metrics
def train_single():
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2,random_state=2018, stratify=train_y)
    model = VDNN().model(embed_glove, maxlen, max_features)
    history = model.fit(X_train, y_train, batch_size=512, epochs=5, validation_data=(X_val, y_val),)
    pred_glove_val_y = model.predict([X_val], batch_size=1024, verbose=1)
    best_threshold = 0
    best_score = 0

    for threshold in [i * 0.01 for i in range(25,50)]:
        score = metrics.f1_score(y_val, (pred_glove_val_y>threshold).astype(int))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    print('best score:%f,best threshold:%f'%(best_score, best_threshold))
    pred_test_y = model.predict(test_X, batch_size=1024, verbose=1)
    pred_test_y = (pred_test_y > best_threshold).astype(int)
    gc.collect()
    K.clear_session()
    tf.reset_default_graph()
    return pred_test_y

In [ ]:
pred_test_y = train_single()

In [ ]:
def Submit(pred_test_y):
    submit = pd.read_csv("../input/sample_submission.csv")
    submit['prediction'] = pred_test_y
    submit.to_csv('submission.csv',index=False)

In [ ]:
KFOLD = 5
SEED = 2018
KFolds = list(StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED).split(train_X, train_y))
train_meta = np.zeros((train_y.shape[0],1))
test_meta = np.zeros((test_X.shape[0],1))
for i, (train_idx, valid_idx) in enumerate(KFolds):
    X_train, X_val, y_train, y_val = train_X[train_idx], train_X[valid_idx], train_y[train_idx], train_y[valid_idx]
    model = LstmAtn().model(embed_glove, maxlen, max_features)
    history = model.fit(X_train, y_train, batch_size=1024, epochs=2, validation_data=(X_val, y_val),)
    pred_val_y = model.predict([X_val], batch_size=1024, verbose=1)
    pred_test_y = model.predict(test_X, batch_size=1024, verbose=1)

    train_meta[valid_idx] = pred_val_y
    test_meta += pred_test_y


In [ ]:
pred_val_y

In [ ]:
test_meta /= KFOLD
best_score,best_threshold = find_threshold(train_y, train_meta)
pred_test_y = (pred_test_y > best_threshold).astype(int)
Submit(pred_test_y)

In [ ]:
def train_kfold(kfold):
    KFolds = list(StratifiedKFold(n_splits=kfold, shuffle=True, random_state=SEED).split(train_X, train_y))
    train_meta = np.zeros(train_y.shape)
    test_meta = np.zeros(test_X.shape[0])
    for i, (train_idx, valid_idx) in enumerate(KFolds):
        X_train, X_val, y_train, y_val = train_X[train_idx], train_X[valid_idx], train_y[train_idx], train_y[valid_idx]
        model = LstmAtn().model(embed_glove, maxlen, max_features)
        history = model.fit(X_train, y_train, batch_size=512, epochs=5, validation_data=(X_val, y_val),)
        pred_val_y = model.predict([X_val], batch_size=1024, verbose=1)
        pred_test_y = model.predict(test_X, batch_size=1024, verbose=1)
        
        train_meta[valid_idx] = pred_val_y
        test_meta += pred_test_y
    test_meat /= kfold